# **Tasks:**
#### 1. Which brewery produces the strongest beers by abv ?
#### 2. If you had to pick 3 beers to recommend to someone, how would you approach the problem ?
#### 3. What are the factors that impacts the quality of beer the most ?
####  4. I enjoy a beer which aroma and appearance matches the beer style. What beer should I buy ?


## Installing pyspark

In [21]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=e40a25828118d142c9ec668c91056b6a932ec1bbba93f5717836f32ab7b5805a
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


## Importing Libraries

In [14]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter

## Creating PySpark Session and importing PySpark Libraries

In [22]:
from pyspark.sql import SparkSession
import pandas as pd
from collections import Counter
from pyspark.sql import functions as f
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

spark

## Importing data

In [24]:
beer_df = pd.read_csv('https://query.data.world/s/xwfkz3u55ocvwpbeaqdqowjmlm6k4t?dws=00000')
sdf = spark.createDataFrame(pd.read_csv('https://query.data.world/s/xwfkz3u55ocvwpbeaqdqowjmlm6k4t?dws=00000'))

In [3]:
df_rows=beer_df.loc[:]
beer_df.info()
beer_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1586614 entries, 0 to 1586613
Data columns (total 13 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   brewery_id          1586614 non-null  int64  
 1   brewery_name        1586599 non-null  object 
 2   review_time         1586614 non-null  int64  
 3   review_overall      1586614 non-null  float64
 4   review_aroma        1586614 non-null  float64
 5   review_appearance   1586614 non-null  float64
 6   review_profilename  1586266 non-null  object 
 7   beer_style          1586614 non-null  object 
 8   review_palate       1586614 non-null  float64
 9   review_taste        1586614 non-null  float64
 10  beer_name           1586614 non-null  object 
 11  beer_abv            1518829 non-null  float64
 12  beer_beerid         1586614 non-null  int64  
dtypes: float64(6), int64(3), object(4)
memory usage: 157.4+ MB


,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
0,10325,Vecchio Birraio,1234817823,1.5,2.0,2.5,stcules,Hefeweizen,1.5,1.5,Sausa Weizen,5.0,47986
1,10325,Vecchio Birraio,1235915097,3.0,2.5,3.0,stcules,English Strong Ale,3.0,3.0,Red Moon,6.2,48213
2,10325,Vecchio Birraio,1235916604,3.0,2.5,3.0,stcules,Foreign / Export Stout,3.0,3.0,Black Horse Black Beer,6.5,48215
3,10325,Vecchio Birraio,1234725145,3.0,3.0,3.5,stcules,German Pilsener,2.5,3.0,Sausa Pils,5.0,47969
4,1075,Caldera Brewing Company,1293735206,4.0,4.5,4.0,johnmichaelsen,American Double / Imperial IPA,4.0,4.5,Cauldron DIPA,7.7,64883


In [4]:
# Checking empty values
for x in beer_df:
  print(f'The number of nan values in {x} is {beer_df[x].isna().sum()}')


The number of nan values in brewery_id is 0
The number of nan values in brewery_name is 15
The number of nan values in review_time is 0
The number of nan values in review_overall is 0
The number of nan values in review_aroma is 0
The number of nan values in review_appearance is 0
The number of nan values in review_profilename is 348
The number of nan values in beer_style is 0
The number of nan values in review_palate is 0
The number of nan values in review_taste is 0
The number of nan values in beer_name is 0
The number of nan values in beer_abv is 67785
The number of nan values in beer_beerid is 0


In [5]:
num=len(beer_df[beer_df['beer_abv'].isna()]['brewery_name'].unique())
brewery_list=beer_df['brewery_name'].unique()
print(f'There are {len(brewery_list)} unique breweries in the dataset.')
print(f'{num} breweries contain nan abv values for some of their beers.')

There are 5743 unique breweries in the dataset.
2779 breweries contain nan abv values for some of their beers.


### One important insight we can derive from this is that the breweries which contain nan beer_abv values are spread out, and 65k/1.5mil is approx 4.3%, so we will not be losing too much information by removing the rows.

In [30]:
df=beer_df.dropna()
sdf = sdf.na.drop()

In [7]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1518478 entries, 0 to 1586613
Data columns (total 13 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   brewery_id          1518478 non-null  int64  
 1   brewery_name        1518478 non-null  object 
 2   review_time         1518478 non-null  int64  
 3   review_overall      1518478 non-null  float64
 4   review_aroma        1518478 non-null  float64
 5   review_appearance   1518478 non-null  float64
 6   review_profilename  1518478 non-null  object 
 7   beer_style          1518478 non-null  object 
 8   review_palate       1518478 non-null  float64
 9   review_taste        1518478 non-null  float64
 10  beer_name           1518478 non-null  object 
 11  beer_abv            1518478 non-null  float64
 12  beer_beerid         1518478 non-null  int64  
dtypes: float64(6), int64(3), object(4)
memory usage: 162.2+ MB


,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
0,10325,Vecchio Birraio,1234817823,1.5,2.0,2.5,stcules,Hefeweizen,1.5,1.5,Sausa Weizen,5.0,47986
1,10325,Vecchio Birraio,1235915097,3.0,2.5,3.0,stcules,English Strong Ale,3.0,3.0,Red Moon,6.2,48213
2,10325,Vecchio Birraio,1235916604,3.0,2.5,3.0,stcules,Foreign / Export Stout,3.0,3.0,Black Horse Black Beer,6.5,48215
3,10325,Vecchio Birraio,1234725145,3.0,3.0,3.5,stcules,German Pilsener,2.5,3.0,Sausa Pils,5.0,47969
4,1075,Caldera Brewing Company,1293735206,4.0,4.5,4.0,johnmichaelsen,American Double / Imperial IPA,4.0,4.5,Cauldron DIPA,7.7,64883


## 1. Which brewery produces the strongest beers by abv ?

### Method 1

In [8]:
maxVal=beer_df.max()
maxVal

<ipython-input-8-4aa1e76cba60>:1: FutureWarning: The default value of numeric_only in DataFrame.max is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  maxVal=beer_df.max()


brewery_id                         28003
review_time                   1326285348
review_overall                       5.0
review_aroma                         5.0
review_appearance                    5.0
beer_style                       Witbier
review_palate                        5.0
review_taste                         5.0
beer_name            葉山ビール (Hayama Beer)
beer_abv                            57.7
beer_beerid                        77317
dtype: object

In [9]:
beer_df.loc[beer_df['beer_abv']==57.7]

,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
12919,6513,Schorschbräu,1316780901,4.0,4.0,4.0,kappldav123,Eisbock,4.0,3.5,Schorschbräu Schorschbock 57%,57.7,73368


As can be seen above, the brewery with the strongest beer in the dataset is ***Schorschbräu***

### Method 2

### Code to show

In [ ]:
brewery_list=df['brewery_name'].unique()
b_dict={}

for x in brewery_list:
  y = df.loc[df['brewery_name']==x]['beer_abv']
  temp=[]
  for z in y:
    temp.append(z)
  b_dict[x]=np.mean(temp)

print(b_dict)

### Output code

In [11]:
sorted_b_dict = sorted(b_dict.items(), key=lambda x:x[1], reverse=True)
counter=0
for x, y in sorted_b_dict:
  counter=counter+1
  print(f'{counter} -> {x}: {y}')
  if (counter>19):
    break

1 -> Schorschbräu: 19.228823529411763
2 -> Shoes Brewery: 15.2
3 -> Rome Brewing Company: 13.84
4 -> Hurlimann Brewery: 13.75
5 -> Alt-Oberurseler Brauhaus: 13.2
6 -> Rascal Creek Brewing Co.: 13.0
7 -> Monks Porter House: 12.466666666666669
8 -> Brasserie Grain d' Orge (Brasserie Jeanne d'Arc SA): 12.445859872611464
9 -> Tugboat Brewing Company: 12.1875
10 -> Rinkuki&#371; Aluas Darykla: 12.0
11 -> United Brands Company: 12.0
12 -> Morgan Street Brewery: 12.0
13 -> Snowy Mountain Brewery: 12.0
14 -> Brauerei Schloss Eggenberg: 11.779681493684786
15 -> Etna Brewery: 11.75
16 -> Nasu Kogen Beer Co. Ltd.: 11.5
17 -> Brasserie Dubuisson Frères sprl: 11.4322191272052
18 -> Kuhnhenn Brewing Company: 11.345839285714286
19 -> Main Street Brewery  / Turoni's Pizza: 11.25
20 -> Zago S.r.l.: 11.0


**Here you can see the top twenty breweries by the beer_abv content.**

## 2. Recommend Three beers to someone.How you would go about it?

One of the ways I thought about was utilizing the review_overall and seeing how different reviewers had different rating systems.
I thought that I should be able to pull the best beers by each reviewer and then collect the most common top 3.

In [12]:
print (len(df['review_profilename'].unique()))
reviewers = df['review_profilename'].unique()

32908


In [13]:
top_beers=[]

for x in reviewers:
  y = df.loc[df['review_profilename']==x][['beer_name' , 'review_overall']]
  y=y.sort_values(by='review_overall', ascending=False)
  y = y[:3]
  top_beers.append(y['beer_name'])

In [15]:
concat_list = [j for i in top_beers for j in i]
c = Counter(concat_list)
c.most_common(3)

[('Pliny The Elder', 650),
 ('Founders Breakfast Stout', 482),
 ("Bell's Hopslam Ale", 460)]

## 3. What are the factors that impacts the quality of beer the most ?

I am using correlation between columns to see what factors impact the overall rating of the beers



In [16]:
new_df=df.drop(['brewery_id', 'review_time', 'beer_beerid'] , axis=1 )
new_df.corr()

<ipython-input-16-672631a4de1d>:2: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  new_df.corr()


,review_overall,review_aroma,review_appearance,review_palate,review_taste,beer_abv
review_overall,1.000000,0.612793,0.498557,0.699020,0.787171,0.138457
review_aroma,0.612793,1.000000,0.559077,0.614924,0.714777,0.332536
review_appearance,0.498557,0.559077,1.000000,0.564555,0.544579,0.263891
review_palate,0.699020,0.614924,0.564555,1.000000,0.732201,0.286667
review_taste,0.787171,0.714777,0.544579,0.732201,1.000000,0.290782
beer_abv,0.138457,0.332536,0.263891,0.286667,0.290782,1.000000


from matplotlib import pyplot as plt
_df_0['review_overall'].plot(kind='hist', bins=20, title='review_overall')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['review_aroma'].plot(kind='hist', bins=20, title='review_aroma')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2['review_appearance'].plot(kind='hist', bins=20, title='review_appearance')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
_df_7.plot(kind='scatter', x='review_palate', y='review_taste', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_8['review_overall'].plot(kind='line', figsize=(8, 4), title='review_overall')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_9['review_aroma'].plot(kind='line', figsize=(8, 4), title='review_aroma')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_10['review_appearance'].plot(kind='line', figsize=(8, 4), title='review_appearance')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_11['review_palate'].plot(kind='line', figsize=(8, 4), title='review_palate')
plt.gca().spines[['top', 'right']].set_visible(False)

Couple of insights that can be derived from the correlation table above: \
1. Taste affects overall rating the most
2. Palate and aroma closely follow
3. Interesting point is that abv does not affect overall rating too much

## 4. I enjoy a beer which aroma and appearance matches the beer style. What beer should I buy ?

In [31]:
from pyspark.sql.types import StructType,StructField, StringType, IntegerType

schema = StructType([ StructField("beer_style",StringType(),True),  StructField("mode(beer_name)",StringType(),True)])
df_test = spark.createDataFrame([("Hefeweizen", "Sierra Nevada Kellerweis Hefeweizen")], schema=schema)

for x in df['beer_style'].unique():
  if (x=='Hefeweizen'):
    continue
  b = sdf.filter(sdf.beer_style==x).select('beer_name', 'brewery_name', 'beer_style').sort('review_aroma', 'review_appearance', ascending=[False, False]).limit(15).groupby('beer_style').agg(f.mode('beer_name')).limit(1)
  df_test = df_test.union(b)

In [33]:
len_=len(df['beer_style'].unique())
df_test.show(truncate=False)

+------------------------------+---------------------------------------------+
|beer_style                    |mode(beer_name)                              |
+------------------------------+---------------------------------------------+
|Hefeweizen                    |Sierra Nevada Kellerweis Hefeweizen          |
|English Strong Ale            |Fuller's 1845                                |
|Foreign / Export Stout        |Lion Stout                                   |
|German Pilsener               |Tannenzäpfle                                 |
|American Double / Imperial IPA|Founders Devil Dancer                        |
|Herbed / Spiced Beer          |Baladin Nora                                 |
|Light Lager                   |Sam Adams Light                              |
|Oatmeal Stout                 |Shakespeare Oatmeal Stout                    |
|American Pale Lager           |Moosehead Lager                              |
|Rauchbier                     |Aecht Schlenkerla Ra